<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [`GALigandDock` Protocol with `pyrosetta.distributed` Using the `beta_cart.wts` Scorefunction](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/08.02-Ligand-Docking-pyrosetta.distributed.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Working With Density](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/11.00-Working-With-Density.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/10.00-Working-With-Symmetry.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Working With Symmetry
Keywords: symmetry, asymmetric, SetupForSymmetryMover, virtual

## Overview
Symmetry is an important concept to learn when working with biomolecules.  When a protein is crystalized, it is in the precense of its symmetrical neighbors - which can be important if testing particular protocols or using crystal density for refinement or full structure building.

Symmetry can also be useful for designing symmetrical structures or large repeating meta-proteins like protein cages. 

### Symmetry In Rosetta
So why do we care if our protein is symmetrical or not when it comes to Rosetta?  Each residue and atom that is loaded into Rosetta takes time to both load, and time to score.  Since scoring can happen thousands of times - even in a short protocol, anything we can do to speed this up becomes important. The most expensive operation in Rosetta is minimization, and by using symmetry - we can reduce the minimization time exponentially by minimizing a single copy instead of ALL copies.  We will get into the details about how this works below.

When we use symmetry in Rosetta - we are basically telling rosetta that the symmetrical partners are 'special', however, the total number of residues is now ALL residues, including symmetrical partners.  Upon setting up symmety in Rosetta, Rosetta will replace the `Conformation` within the pose with a **Symmetrical** version, called the `SymmetricConformation`.  If you know anything about classes, this `SymmetricConformation` is derived from the actual `Conformation` object, but contains extra information about the pose and some functions are replaced. 

### Symmetric Scoring and Moving
Ok, so now lets assume that we have our symmetric pose.  Now what?  Well, the symmetric copies are all tied to their real counterparts.  Once you move a chain, residue, or atom by packing or minimization, the symmetric copies of that residue are all moved in the same way. 

Cool.  But what about scoring?  Scoring works very similarly - instead of scoring each and every residue in our pose, Rosetta will score just our assymetric unit, and multiply that out to the number of symmetric copies we have.  Intelligently, Rosetta will also figure out the symmetric interfaces that arise from the interactions of our assymetric unit to the symmetric copies and score them appropriately.

### Symmetry-aware movers
Most of our common movers are symmetry-aware.  At one point there were different symmetric and non-symmetric versions of particular code, such as MinMover and PackRotamersMover.  Now though, Rosetta will automatically use the pose to figure out what needs to be done. You should seek original documentation (and contact the author if not explicit) to make sure that an uncommon protocol you are using is symmetry-aware.

## Documentation
More information on RosettaSymmetry can be found in the following places:
- https://www.rosettacommons.org/docs/latest/rosetta_basics/structural_concepts/symmetry
- https://www.rosettacommons.org/demos/latest/tutorials/Symmetry/Symmetry
- https://www.rosettacommons.org/docs/latest/application_documentation/utilities/make-symmdef-file
- https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/SetupForSymmetryMover
- https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/ExtractAsymmetricUnitMover


In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

Here, we will use a few specific options. The first three options make Rosetta a bit more robust to input structures.  The `-load_PDB_components` cannot be used with glycans, unfortunately, and our structure has a few very important glycans.  Finally, we load a bunch of glycan-specific options, which we will cover in the next tutorial.

In [1]:
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *
import os

init('-ignore_unrecognized_res -load_PDB_components false -ignore_zero_occupancy false @inputs/glycan_flags')

PyRosetta-4 2019 [Rosetta PyRosetta4.Release.python36.mac 2019.39+release.93456a567a8125cafdf7f8cb44400bc20b570d81 2019-09-26T14:24:44] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Reading fconfig.../Users/jadolfbr/.rosetta/flags/common
core.init: 
core.init: 
core.init: Rosetta version: PyRosetta4.Release.python36.mac r233 2019.39+release.93456a567a8 93456a567a8125cafdf7f8cb44400bc20b570d81 http://www.pyrosetta.org 2019-09-26T14:24:44
core.init: command: PyRosetta -ignore_unrecognized_res -load_PDB_components false -ignore_zero_occupancy false @inputs/glycan_flags -database /Users/jadolfbr/Library/Python/3.6/lib/python/site-packages/pyrosetta-2019.39+release.93456a567a8-py3.6-macosx-10.6-intel.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1544006277 

## Creating a SymDef file

Here, we will start with how to create a basic symdef file for cyrstal symmetry.  Note that there are ways to do this without a symdef file, but these do not currently work for glycan structures, which we will be using here. 

The `make_symdef_file.pl` file is within Rosetta3. To use it, you will need to download and licence Rosetta3.  The code is in the `Rosetta/main/src/apps/public` directory. In the interest of reducing code drift, this file is NOT included in the tutorial directory as we may then have version drift.  

If you have done this, we can use the following command to create the symdef file.  Here, the radius of symmetrical partners is 12A, which is certainly fairly large, but produces a very well represented crystal.

In [3]:
pdb = "inputs/1jnd.pdb"
base_cmd = f'cd inputs && make_symmdef_file.pl -r 12 -m CRYST -p  {pdb}.pdb > {pdb}_crys.symm && cd -'
print(base_cmd)

cd inputs && make_symmdef_file.pl -r 12 -m CRYST -p  inputs/1jnd.pdb.pdb > inputs/1jnd.pdb_crys.symm && cd -


Use this base command and the `os.system(cmd)` function to run the code or use the provided symdef file. 

In [3]:
os.system('cp inputs/1jnd_crys.symm .')

0

Take a look at the symmetrized structure in pymol (`inputs/1jnd_symm.pdb`).  What would happen if we increased the radius to 24 instead of 12?

## Setup a Symmetrized Pose

Here, we will run a basic Rosetta protocol with symmetry.  There are much more complicated things you can do with symmetry, but for now, we just want to symmetrically pack the protein.  Please see the docs for more on symmetry.  The full Rosetta C++ tutorial for symmetry is a great place to go from here: - https://www.rosettacommons.org/demos/latest/tutorials/Symmetry/Symmetry 

Lets first create a pose, and then use the `SetupForSymmetryMover` on the pose. Note this is an unrefined input structure.  This is so that minmover will actually do something.  A pareto-optimal refined structure can be found in the inputs as `1jnd_refined.pdb.gz`

In [37]:
p = pose_from_pdb('inputs/1jnd.pdb')
original = p.clone()

core.import_pose.import_pose: File 'inputs/1jnd.pdb' automatically determined to be of type PDB
core.io.pdb.pdb_reader: Parsing 82 .pdb records with unknown format to search for Rosetta-specific comments.
core.io.util: Automatic glycan connection is activated.
core.io.util: Start reordering residues.
core.io.util: Corrected glycan residue order (internal numbering): [401, 402, 403, 404]
core.io.util: 
core.io.pose_from_sfr.PoseFromSFRBuilder: Setting chain termination for 404
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] Glc401 has an unfavorable ring conformation; the coordinates for this input structure may have been poorly assigned.
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] Glc402 has an unfavorable ring conformation; the coordinates for this input structure may have been poorly assigned.
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] Man403 has an unfavorable ring conformation; the coordinates for this input structure may have been poorly assigned.
core.i

In [38]:
p.total_residue()

404

In [39]:
type(p.conformation())

pyrosetta.rosetta.core.conformation.Conformation

In [40]:
symmetrize = rosetta.protocols.symmetry.SetupForSymmetryMover("1jnd_crys.symm")
symmetrize.apply(p)

core.conformation.symmetry.SymmData: [ WARNING ] Setting weight of master jump ( jump-id=1 ) to 1.0 (was undefined)
core.conformation.symmetry.SymmData: [ WARNING ] Setting jump_group JUMPGROUP1: [master 1]  6:0  7:0  2:0  3:0  4:0  5:0  8:0  9:0
core.conformation.symmetry.SymmData: [ WARNING ] Setting weight of master jump ( jump-id=10 ) to 1.0 (was undefined)
core.conformation.symmetry.SymmData: [ WARNING ] Setting jump_group JUMPGROUP2: [master 10]  20:0  22:0  12:0  14:0  16:0  18:0  24:0  26:0
core.conformation.symmetry.SymmData: symmetry name: 1jnd_P_32_2_1
core.conformation.symmetry.SymmData: number of subunits: 9
core.conformation.symmetry.SymmData: number of interfaces: 1
core.conformation.symmetry.SymmData: score subunit number: VRT_0_0_0_0
core.conformation.symmetry.SymmData: anchor the subunits at residue: 1
core.conformation.symmetry.SymmData:  Virtual coordinate system VRT_0_0_0_0
core.conformation.symmetry.SymmData: x: 1 0 0
core.conformation.symmetry.SymmData: y: 0 1 0


core.conformation.symmetry.SymmData: Jump JUMP_3_1_0_0_to_subunit VRT_3_1_0_0_base SUBUNIT
core.conformation.symmetry.SymmData: Jump JUMP_4_2_1_0 VRT_0_0_0_0 VRT_4_2_1_0
core.conformation.symmetry.SymmData: Jump JUMP_4_2_1_0_to_com VRT_4_2_1_0 VRT_4_2_1_0_base
core.conformation.symmetry.SymmData: Jump JUMP_4_2_1_0_to_subunit VRT_4_2_1_0_base SUBUNIT
core.conformation.symmetry.SymmData: Jump JUMP_4_2_1_n1 VRT_0_0_0_0 VRT_4_2_1_n1
core.conformation.symmetry.SymmData: Jump JUMP_4_2_1_n1_to_com VRT_4_2_1_n1 VRT_4_2_1_n1_base
core.conformation.symmetry.SymmData: Jump JUMP_4_2_1_n1_to_subunit VRT_4_2_1_n1_base SUBUNIT
core.conformation.symmetry.SymmData: Jump JUMP_5_0_1_0 VRT_0_0_0_0 VRT_5_0_1_0
core.conformation.symmetry.SymmData: Jump JUMP_5_0_1_0_to_com VRT_5_0_1_0 VRT_5_0_1_0_base
core.conformation.symmetry.SymmData: Jump JUMP_5_0_1_0_to_subunit VRT_5_0_1_0_base SUBUNIT
core.conformation.symmetry.SymmData: Jump JUMP_5_1_1_0 VRT_0_0_0_0 VRT_5_1_1_0
core.conformation.symmetry.SymmData: Jum

In [41]:
print(p.total_residue())
print(type(p.conformation()))

3654
<class 'pyrosetta.rosetta.core.conformation.symmetry.SymmetricConformation'>


How many symmetric copies do we have in our pose?
How do the scores compare for our original pose and our symmetrized version?

Now lets use some of the functionality to understand how this all works. We can use the `SymetricInfo` object that is part of the `SymmetricConformation` to get at some info. Lets take a look at all residues and find the assymetric unit residues and equivalent residues for the rest.

In [42]:
print("AssymUnit? equivalent_res")
sym_info = p.conformation().Symmetry_Info()
for i in range(1, p.size()+1):
    print(i, sym_info.bb_is_independent(i), sym_info.bb_follows(i))

AssymUnit? equivalent_res
1 True 0
2 True 0
3 True 0
4 True 0
5 True 0
6 True 0
7 True 0
8 True 0
9 True 0
10 True 0
11 True 0
12 True 0
13 True 0
14 True 0
15 True 0
16 True 0
17 True 0
18 True 0
19 True 0
20 True 0
21 True 0
22 True 0
23 True 0
24 True 0
25 True 0
26 True 0
27 True 0
28 True 0
29 True 0
30 True 0
31 True 0
32 True 0
33 True 0
34 True 0
35 True 0
36 True 0
37 True 0
38 True 0
39 True 0
40 True 0
41 True 0
42 True 0
43 True 0
44 True 0
45 True 0
46 True 0
47 True 0
48 True 0
49 True 0
50 True 0
51 True 0
52 True 0
53 True 0
54 True 0
55 True 0
56 True 0
57 True 0
58 True 0
59 True 0
60 True 0
61 True 0
62 True 0
63 True 0
64 True 0
65 True 0
66 True 0
67 True 0
68 True 0
69 True 0
70 True 0
71 True 0
72 True 0
73 True 0
74 True 0
75 True 0
76 True 0
77 True 0
78 True 0
79 True 0
80 True 0
81 True 0
82 True 0
83 True 0
84 True 0
85 True 0
86 True 0
87 True 0
88 True 0
89 True 0
90 True 0
91 True 0
92 True 0
93 True 0
94 True 0
95 True 0
96 True 0
97 True 0
98 True 0
99 

892 False 84
893 False 85
894 False 86
895 False 87
896 False 88
897 False 89
898 False 90
899 False 91
900 False 92
901 False 93
902 False 94
903 False 95
904 False 96
905 False 97
906 False 98
907 False 99
908 False 100
909 False 101
910 False 102
911 False 103
912 False 104
913 False 105
914 False 106
915 False 107
916 False 108
917 False 109
918 False 110
919 False 111
920 False 112
921 False 113
922 False 114
923 False 115
924 False 116
925 False 117
926 False 118
927 False 119
928 False 120
929 False 121
930 False 122
931 False 123
932 False 124
933 False 125
934 False 126
935 False 127
936 False 128
937 False 129
938 False 130
939 False 131
940 False 132
941 False 133
942 False 134
943 False 135
944 False 136
945 False 137
946 False 138
947 False 139
948 False 140
949 False 141
950 False 142
951 False 143
952 False 144
953 False 145
954 False 146
955 False 147
956 False 148
957 False 149
958 False 150
959 False 151
960 False 152
961 False 153
962 False 154
963 False 155
964 Fals

1976 False 360
1977 False 361
1978 False 362
1979 False 363
1980 False 364
1981 False 365
1982 False 366
1983 False 367
1984 False 368
1985 False 369
1986 False 370
1987 False 371
1988 False 372
1989 False 373
1990 False 374
1991 False 375
1992 False 376
1993 False 377
1994 False 378
1995 False 379
1996 False 380
1997 False 381
1998 False 382
1999 False 383
2000 False 384
2001 False 385
2002 False 386
2003 False 387
2004 False 388
2005 False 389
2006 False 390
2007 False 391
2008 False 392
2009 False 393
2010 False 394
2011 False 395
2012 False 396
2013 False 397
2014 False 398
2015 False 399
2016 False 400
2017 False 401
2018 False 402
2019 False 403
2020 False 404
2021 False 1
2022 False 2
2023 False 3
2024 False 4
2025 False 5
2026 False 6
2027 False 7
2028 False 8
2029 False 9
2030 False 10
2031 False 11
2032 False 12
2033 False 13
2034 False 14
2035 False 15
2036 False 16
2037 False 17
2038 False 18
2039 False 19
2040 False 20
2041 False 21
2042 False 22
2043 False 23
2044 False 2

2882 False 54
2883 False 55
2884 False 56
2885 False 57
2886 False 58
2887 False 59
2888 False 60
2889 False 61
2890 False 62
2891 False 63
2892 False 64
2893 False 65
2894 False 66
2895 False 67
2896 False 68
2897 False 69
2898 False 70
2899 False 71
2900 False 72
2901 False 73
2902 False 74
2903 False 75
2904 False 76
2905 False 77
2906 False 78
2907 False 79
2908 False 80
2909 False 81
2910 False 82
2911 False 83
2912 False 84
2913 False 85
2914 False 86
2915 False 87
2916 False 88
2917 False 89
2918 False 90
2919 False 91
2920 False 92
2921 False 93
2922 False 94
2923 False 95
2924 False 96
2925 False 97
2926 False 98
2927 False 99
2928 False 100
2929 False 101
2930 False 102
2931 False 103
2932 False 104
2933 False 105
2934 False 106
2935 False 107
2936 False 108
2937 False 109
2938 False 110
2939 False 111
2940 False 112
2941 False 113
2942 False 114
2943 False 115
2944 False 116
2945 False 117
2946 False 118
2947 False 119
2948 False 120
2949 False 121
2950 False 122
2951 False 

Which residues are our original pose residues?  Note that the final residues are called `Virtual` residues.  Virtual residues are not scored.  They have coordinates, and can move, but simply result in a score of zero.  They are useful in some contexts to hide a part of the pose from the scoring machinery, and there are movers that can change residues to and from virtual.  In this case, they are used for the FoldTree - in order to allow refinement of the full crystal environment.  They allow relative movement of each subunit relative to each other.  There are two virtual residues for each subunit

In [43]:
print(p.residue(3654))

Residue 3654: VRT (XXX, X):
Base: VRT
 Properties: LIGAND VIRTUAL_RESIDUE
 Variant types:
 Main-chain atoms:
 Backbone atoms:  
 Side-chain atoms: ORIG  X    Y  
Atom Coordinates:
  ORIG: 126.048, 44.0754, 19.9707 (virtual)
   X  : 127.048, 44.0754, 19.9707 (virtual)
   Y  : 126.048, 43.0754, 19.9707 (virtual)
Mirrored relative to coordinates in ResidueType: FALSE



In [44]:
print("Total Subunits:", (3654-18)/404)
print("Total Subunits:", sym_info.subunits())

Total Subunits: 9.0
Total Subunits: 9


In [45]:
score = get_score_function()
print(score(original))
print(score(p))

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: The -include_sugars flag was used with no sugar_bb weight set in the weights file.  Setting sugar_bb weight to 1.0 by default.
-531.2534669946483
-1132.7274634504747


## Running Protocols with Symmetry

Now, lets try running a minimization with symmetry on.  

In [54]:
mm = MoveMap()
mm.set_bb(True)
mm.set_chi(True)
minmover = rosetta.protocols.minimization_packing.MinMover()
minmover.score_function(score)
minmover.set_movemap(mm)
minmover.apply(p)

In [55]:
score(p)

-1147.371462425749

How does our pose look?  For being such a large pose, how was the speed of minimization? 

How does this compare to our refined pose?  Try to copy a subunit to a new object in PyMol.  Then use the align command to align it to our assymetric unit.  What is the RMSD?

Now lets pack with our symmetric structure. 

In [68]:
from rosetta.core.pack.task import *
from rosetta.core.pack.task.operation import *

packer = PackRotamersMover()
tf = rosetta.core.pack.task.TaskFactory()

tf.push_back(RestrictToRepacking())
tf.push_back(IncludeCurrent())
packer.task_factory(tf)

p = original.clone()
symmetrize.apply(p)

core.conformation.symmetry.util: =================== SYM FOLD TREE, jump notation: =symfixed= *indep* #symdof# jump[=follows] ========================
VRT_0_0_0_0(3637)
|----#j10#--->VRT_0_0_0_0_base(3638)----#j1#------>1:Sub1A(1-400)
|----=j11=--->VRT_1_1_0_n1(3643)---j12=10--->VRT_1_1_0_n1_base(3644)----j2=1---->1213:Sub4A(1213-1612)
|----=j13=--->VRT_2_1_1_0(3645)---j14=10--->VRT_2_1_1_0_base(3646)----j3=1---->1617:Sub5A(1617-2016)
|----=j15=--->VRT_3_0_n1_0(3647)---j16=10--->VRT_3_0_n1_0_base(3648)----j4=1---->2021:Sub6A(2021-2420)
|----=j17=--->VRT_3_1_0_0(3649)---j18=10--->VRT_3_1_0_0_base(3650)----j5=1---->2425:Sub7A(2425-2824)
|----=j19=--->VRT_4_2_1_0(3639)---j20=10--->VRT_4_2_1_0_base(3640)----j6=1---->405:Sub2A(405-804)
|----=j21=--->VRT_4_2_1_n1(3641)---j22=10--->VRT_4_2_1_n1_base(3642)----j7=1---->809:Sub3A(809-1208)
|----=j23=--->VRT_5_0_1_0(3651)---j24=10--->VRT_5_0_1_0_base(3652)----j8=1---->2829:Sub8A(2829-3228)
\----=j25=--->VRT_5_1_1_0(3653)---j26=10--->VRT_5_1_1_0_b

In [69]:
packer.apply(p)

protocols.minimization_packing.PackRotamersMover: [ WARNING ] undefined ScoreFunction -- creating a default one
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: The -include_sugars flag was used with no sugar_bb weight set in the weights file.  Setting sugar_bb weight to 1.0 by default.
core.pack.rotamer_set.RotamerSet_: [ WARNING ] including current in order to get at least 1 rotamer !!!!!! 180 ASN:N-glycosylated
core.pack.pack_rotamers: built 5259 rotamers at 404 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph


In [70]:
print("packed", score(p))

packed -1960.9189300248413


## Conclusions

Symmetry is a useful tool in the Rosetta Library.  There are also selectors and movers that you may find useful, such as the `AsymmetricUnitSelector` in `rosetta.core.select.residue_selectors` and the `ExtractAsymmetricUnitMover`, which will give you back just the single subunit, without any asymetric partners, and the `ExtractAsymmetricPoseMover`, which will remove 'symmetry' information and give you back a pose with all the subunits. The later of these can be found by importing `rosetta.protocols.symmetry`.

Note that not ALL protocols will respect symmetry - so please check the original documentation to see if symmetry is supported.  If you are unsure, please email the developer.

**Chapter contributors:**

- Jared Adolf-Bryfogle (Scripps; Institute for Protein Innovation)

<!--NAVIGATION-->
< [`GALigandDock` Protocol with `pyrosetta.distributed` Using the `beta_cart.wts` Scorefunction](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/08.02-Ligand-Docking-pyrosetta.distributed.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Working With Density](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/11.00-Working-With-Density.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/10.00-Working-With-Symmetry.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>